In [ ]:
!pip install ultralytics

In [2]:
from ultralytics import YOLO
model = YOLO("bestv8.pt")

In [3]:
import cv2
# Function to run the model on an image and display results
def detect_on_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    
    # Run detection
    results = model(img)
    
    # Visualize results
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])  # Convert tensor to float
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 1)
            cv2.putText(img, f'{conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    # Display the image
    cv2.imshow('YOLOv8 Detection', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

detect_on_image('Large-crowd-of-people-014.jpg')



0: 384x640 78 Persons, 359.3ms
Speed: 7.0ms preprocess, 359.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


In [4]:
#Density Map

import cv2
import numpy as np
import random
import time
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load the YOLO model
model = YOLO("bestv8.pt")

# Initialize DeepSORT
deepsort = DeepSort(max_age=15, n_init=3, nms_max_overlap=1.0, max_iou_distance=0.7)

def get_random_color():
    return [random.randint(0, 255) for _ in range(3)]

def process_frame(frame, frame_count, detections, track_colors, density_map, threshold, warning_shown):
    # Run detection every 3 frames
    if frame_count % 3 == 0:
        results = model(frame, conf=0.3, iou=0.5)  # Adjust the confidence and IoU thresholds here
        detections.clear()
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                w, h = x2 - x1, y2 - y1
                conf = float(box.conf[0])
                if conf > 0.3:  # Ensure the confidence threshold is met
                    detections.append(((x1, y1, w, h), conf))
    
    # Update tracker with current frame detections
    tracks = deepsort.update_tracks(detections, frame=frame)
    
    # Count people and visualize results
    num_people = 0
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        bbox = track.to_ltwh()
        x1, y1, w, h = map(int, bbox)
        x2, y2 = x1 + w, y1 + h
        
        if track_id not in track_colors:
            track_colors[track_id] = get_random_color()
            
        color = track_colors[track_id]
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        num_people += 1
        density_map[y1:y2, x1:x2] += 1

    # Normalize density map for visualization
    density_norm = cv2.normalize(density_map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    density_color_map = cv2.applyColorMap(density_norm, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(frame, 0.7, density_color_map, 0.3, 0)

    # Display the count on the frame
    cv2.putText(overlay, f'Current People: {num_people}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Display warning if count exceeds threshold
    if num_people > threshold:
        cv2.putText(overlay, f'WARNING: Crowd exceeds {threshold}!', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        warning_shown = True
    elif warning_shown:
        warning_shown = False

    return overlay, warning_shown

def detect_on_video(video_path, threshold=10):
    # Open the video file or capture device
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return
    
    # Get video details for saving processed video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object
    out_video_path = video_path.replace('.mp4', '_processed.mp4')
    out_video = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_fps, (frame_width, frame_height))
    
    # Initialize variables for counting people and warning status
    total_people = 0
    start_time = time.time()
    warning_shown = False
    
    # Initialize density map
    density_map = np.zeros((frame_height, frame_width), dtype=np.float32)
    
    # Store unique colors for each track ID
    track_colors = {}
    
    # Frame processing loop
    frame_count = 0
    detections = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        overlay, warning_shown = process_frame(frame, frame_count, detections, track_colors, density_map, threshold, warning_shown)

        # Display the frame with density map overlay
        cv2.imshow('YOLOv8 Detection with DeepSORT and Density Mapping', overlay)
        
        # Write the frame with overlays to the output video
        out_video.write(overlay)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_count += 1

    # Release the video capture and writer, and destroy all OpenCV windows
    cap.release()
    out_video.release()
    cv2.destroyAllWindows()

# Example usage
detect_on_video('people_walking.mp4', threshold=40)  # Change to '0' for webcam 


0: 384x640 41 Persons, 294.0ms
Speed: 4.0ms preprocess, 294.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 41 Persons, 329.7ms
Speed: 4.1ms preprocess, 329.7ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 40 Persons, 328.1ms
Speed: 4.1ms preprocess, 328.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 Persons, 290.9ms
Speed: 2.1ms preprocess, 290.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 Persons, 341.4ms
Speed: 5.6ms preprocess, 341.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 Persons, 275.0ms
Speed: 2.4ms preprocess, 275.0ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 Persons, 283.2ms
Speed: 2.0ms preprocess, 283.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 Persons, 267.7ms
Speed: 2.1ms preprocess, 267.7ms inference, 0.0ms postproc

In [9]:
import cv2
import numpy as np
import random
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load the YOLO model
model = YOLO("bestv8.pt")

# Initialize DeepSORT
deepsort = DeepSort(max_age=15, n_init=3, nms_max_overlap=1.0, max_iou_distance=0.7)

def get_random_color():
    return [random.randint(0, 255) for _ in range(3)]

def process_frame(frame, frame_count, detections, track_colors, threshold, warning_shown):
    if frame_count % 5 == 0:  # Run detection every 5 frames
        resized_frame = cv2.resize(frame, (640, 640))  # Resize for faster processing
        results = model(resized_frame, conf=0.3, iou=0.5)
        detections.clear()
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                w, h = x2 - x1, y2 - y1
                conf = float(box.conf[0])
                if conf > 0.3:
                    scale_w = frame.shape[1] / resized_frame.shape[1]
                    scale_h = frame.shape[0] / resized_frame.shape[0]
                    x1, x2 = int(x1 * scale_w), int(x2 * scale_w)
                    y1, y2 = int(y1 * scale_h), int(y2 * scale_h)
                    detections.append(((x1, y1, x2 - x1, y2 - y1), conf))

    tracks = deepsort.update_tracks(detections, frame=frame)
    num_people = 0
    centers = []

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        bbox = track.to_ltwh()
        x1, y1, w, h = map(int, bbox)
        x2, y2 = x1 + w, y1 + h

        if track_id not in track_colors:
            track_colors[track_id] = get_random_color()

        color = track_colors[track_id]
        center_x, center_y = x1 + w // 2, y1 + h // 2
        centers.append((center_x, center_y))

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        num_people += 1

    for i, center_i in enumerate(centers):
        for j, center_j in enumerate(centers):
            if i != j and np.linalg.norm(np.array(center_i) - np.array(center_j)) < 50:
                cv2.circle(frame, center_i, 5, (0, 0, 255), -1)
                cv2.circle(frame, center_j, 5, (0, 0, 255), -1)

    cv2.putText(frame, f'Current People: {num_people}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    if num_people > threshold:
        cv2.putText(frame, f'WARNING: Crowd exceeds {threshold}!', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        warning_shown = True
    else:
        warning_shown = False

    return frame, warning_shown

def detect_on_video(video_path, threshold=10):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_fps = int(cap.get(cv2.CAP_PROP_FPS))
    out_video_path = 'processed_video.mp4'
    out_video = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_fps, (frame_width, frame_height))

    frame_count = 0
    detections = []
    track_colors = {}
    warning_shown = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        overlay, warning_shown = process_frame(frame, frame_count, detections, track_colors, threshold, warning_shown)
        out_video.write(overlay)

        # Display the frame
        cv2.imshow('Frame', overlay)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    out_video.release()
    cv2.destroyAllWindows()

    print("Video processing complete. Processed video saved as 'processed_video.mp4'")

# Example usage
detect_on_video('people_walking.mp4', threshold=40)



0: 640x640 41 Persons, 469.1ms
Speed: 6.9ms preprocess, 469.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 40 Persons, 554.5ms
Speed: 4.5ms preprocess, 554.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 41 Persons, 574.1ms
Speed: 4.7ms preprocess, 574.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 42 Persons, 503.2ms
Speed: 4.1ms preprocess, 503.2ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 41 Persons, 453.5ms
Speed: 4.0ms preprocess, 453.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 41 Persons, 553.1ms
Speed: 4.0ms preprocess, 553.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 42 Persons, 562.0ms
Speed: 3.6ms preprocess, 562.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 42 Persons, 558.6ms
Speed: 4.0ms preprocess, 558.6ms inference, 1.0ms postproc